In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
dt=pd.read_csv('amazon_product.csv')
data = dt.drop(['Unnamed: 0','product_url','product_original_price' ,'product_availability' ,'product_photo', 'climate_pledge_friendly', 'sales_volume', 'delivery', 'unit_price', 'unit_count','asin'], axis=1)
data['product_star_rating'] = pd.to_numeric(data['product_star_rating'], errors='coerce')
data['product_star_rating']=data['product_star_rating'].fillna(data['product_star_rating'].median())

def convert_dollar_number(dollar):
    return float(dollar.replace('$',''))

# data['product_price'] = data['product_price'].apply(convert_dollar_number)
data['product_minimum_offer_price'] = data['product_minimum_offer_price'].apply(convert_dollar_number)
data['product_price'] = data['product_price'].apply(convert_dollar_number)
print(data.head())

                                       product_title  product_price currency  \
0  Moto G Play 2023 3-Day Battery Unlocked Made f...          99.99      USD   
1  SAMSUNG Galaxy A15 5G (SM-156M/DSN), 128GB 6GB...         149.74      USD   
2  Total by Verizon | Samsung Galaxy A03s | Locke...          49.88      USD   
3  Google Pixel 7a - Unlocked Android Cell Phone ...         335.00      USD   
4  SAMSUNG Galaxy A15 5G A Series Cell Phone, 128...         199.99      USD   

   product_star_rating  product_num_ratings  product_num_offers  \
0                  4.0                 2929                  10   
1                  4.2                  135                   8   
2                  3.9                  205                   1   
3                  4.3                 2248                  30   
4                  4.1                  423                   2   

   product_minimum_offer_price  is_best_seller  is_amazon_choice  is_prime  \
0                        64.89        